# Combinations API

The Combinations API in Hypster allows you to generate and query all possible combinations of hyperparameters defined using `hp.select()` in your configuration. This is particularly useful for hyperparameter tuning and exploring different model configurations.

## Generating Combinations

### Using `get_combinations()`

The `get_combinations()` method generates all possible combinations of hyperparameters for a given configuration, but only for parameters defined with `hp.select()`.

Example:

In [2]:
from hypster import HP, config


@config
def model_config(hp: HP):
    model_type = hp.select(["cnn", "rnn"], default="cnn")
    if model_type == "cnn":
        num_layers = hp.select([3, 5], default=3)
    else:
        cell_type = hp.select(["lstm", "gru"], default="lstm")

    learning_rate = hp.number(default=0.001)
    model_name = hp.text(default="my_model")


# Generate combinations
combinations = model_config.get_combinations()
combinations

[{'model_type': 'cnn', 'num_layers': 3},
 {'model_type': 'cnn', 'num_layers': 5},
 {'model_type': 'rnn', 'cell_type': 'lstm'},
 {'model_type': 'rnn', 'cell_type': 'gru'}]

Note that `learning_rate` and `model_name` are not included in the combinations as they are not defined using `hp.select()`.

### Nested Configurations

For nested configurations using `hp.propagate()`, the combinations will include the nested structure:


In [3]:
from hypster import HP, config, save

save(model_config, "model_config.py")


@config
def training_config(hp: HP):
    import hypster

    batch_size = hp.select([32, 64], default=32)
    num_epochs = hp.number(default=100)
    model_config = hypster.load("model_config.py")
    model_params = hp.propagate(model_config, name="model")


# Generate combinations
combinations = training_config.get_combinations()
combinations

[{'batch_size': 32, 'model.model_type': 'cnn', 'model.num_layers': 3},
 {'batch_size': 32, 'model.model_type': 'cnn', 'model.num_layers': 5},
 {'batch_size': 32, 'model.model_type': 'rnn', 'model.cell_type': 'lstm'},
 {'batch_size': 32, 'model.model_type': 'rnn', 'model.cell_type': 'gru'},
 {'batch_size': 64, 'model.model_type': 'cnn', 'model.num_layers': 3},
 {'batch_size': 64, 'model.model_type': 'cnn', 'model.num_layers': 5},
 {'batch_size': 64, 'model.model_type': 'rnn', 'model.cell_type': 'lstm'},
 {'batch_size': 64, 'model.model_type': 'rnn', 'model.cell_type': 'gru'}]

## Querying Combinations

The `query_combinations()` function allows you to filter the generated combinations based on specific criteria.

### Using `query_combinations()`


In [4]:
from hypster import query_combinations

filtered_combinations = query_combinations(combinations, {"model.model_type": "cnn", "batch_size": 64})
filtered_combinations

[{'batch_size': 64, 'model.model_type': 'cnn', 'model.num_layers': 3},
 {'batch_size': 64, 'model.model_type': 'cnn', 'model.num_layers': 5}]

```{warning}
`get_combinations()` runs your config code multiple times to dynamically explore the options space. Make sure your configuration code runs fast and doesn't incur any charges.  
```